<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js103_solveDiracEquation1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js103_solveDiracEquation1D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.12.08 created,  last updated on 2024.12.27
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js103_solveDiracEquation1D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js103_solveDiracEquation1D
//    Copyright(C) 2018-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2018.06.20 created, last updated on 2020.12.15
//    ver 0.0.1  2021.09.28 v1, last updated on 2021.09.28
//    ver 0.0.2  2021.11.07 v2, last updated on 2021.11.07
//    ver 0.0.3  2023.05.13 v3, last updated on 2023.09.10
//
// -------------------- time dependent Kohn-Sham 1D : LDA + spin
//
//  Jeffrey Yepez; arXiv quant-ph/0210093v1 11 Oct. 2002
//    "An efficient and accurate algorithm for the Dirac equation"
//
//    Dirac equation becomes appoximately eq(24) when dt is small ( error term Oder(dr^2,dt) )
//      eq(24) --> in 1D case (alpha=psi[0], beta=psi[1], mue=psi[2], nue=psi[3])
//      m c^2/hBar = c^2 in au (m=1,hBar=1)
//      dt psi[0] = c^2 (dx*-d{psi[1]} + i psi[2]), where d{psi[1]} = (psi[1][i+1]-psi[1][i-1])/(2*dx)
//      dt psi[1] = c^2 (dx*-d{psi[0]} + i psi[3])
//      dt psi[2] = c^2 (dx* d{psi[3]} + i psi[0])
//      dt psi[3] = c^2 (dx* d{psi[2]} + i psi[1])
//
//    initial condition: |psi[0]|=1,others=0, Gaussianwave, momentum=10.0au
//
// --------------------
*/

const solveDiracEquation1D = (function(){ // ====================  solveDiracEquation1D Module  ====================

	// au: atomic unit hBar=1,e=1,me=1,a0=1
	const g_auLength = 5.29177211e-11;		// (m) 1(au) = g_auLength (m)
	const g_auTime = 2.418884326e-17;		// (s) 1(au) = g_auTime (s)
	const g_auEnergy = 4.35974465e-18;		// (J) 1(au) = g_auEnergy (J)
	const g_au2eV = 27.211386;				// (eV) 1(au) = 27.211386 (eV)

	let g_NNx = 540;						// number of space division
	let g_sysTime = 0.0;					// (au) system time in the module
	let g_dx = 0.05;						// (au) space division
	let g_dt = 1.0e-7;						// (au) time step
	let g_px0 = 10.0;						// (au) momentum

	const g_psi = dim3( 4, g_NNx, 2 );		// wave function psi[0:alpha|1:beta|2:mue|3:nue][ix][0:Re|1:Im]
	const g_vv = [];						// external potential

	function dim3( ni, nj, nk ) {
		let a=[];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( momentum ) {

		g_sysTime = 0.0;
		g_px0 = momentum;
		setDiracWave(10.0, 1.0, g_px0); //(wavePos, waveWidth, momentum)
		setPotential(); // free space
	}

	function setDiracWave(wavePos, waveWidth, momentum) {
		const nnx=g_NNx, psi=g_psi;

		const a = Math.pow(2.0*Math.PI*waveWidth*waveWidth,-0.25);
		for (let i=1; i<nnx-1; i++) {
			const x = i*g_dx;
			const phAb = a*Math.exp(-((x-wavePos)/(2.0*waveWidth))*((x-wavePos)/(2.0*waveWidth)));
			const phPh = momentum*x;
			psi[0][i][0] = phAb*Math.cos(phPh);
			psi[0][i][1] = phAb*Math.sin(phPh);
		}
		psi[0][0][0] = 0.0;
		psi[0][0][1] = 0.0;
		psi[0][nnx-1][0] = 0.0;
		psi[0][nnx-1][1] = 0.0;

		for (let i=0; i<nnx; i++) {
			psi[1][i][0] = 0.0; psi[1][i][1] = 0.0;
			psi[2][i][0] = 0.0; psi[2][i][1] = 0.0;
			psi[3][i][0] = 0.0; psi[3][i][1] = 0.0;
		}

		const sqr = Math.sqrt(normDirac());
		for (let i=1; i<nnx-1; i++) {
			psi[0][i][0] = psi[0][i][0]/sqr;
			psi[0][i][1] = psi[0][i][1]/sqr;
		}
	}

	function setPotential() {
		const nnx=g_NNx;

		for (let i=0; i<nnx; i++) {
			g_vv[i] = 0.0;
		}
	}


	// --------------------  time evolution  --------------------

	function timeEvolution(nTimes) {

		for (let i=0; i<nTimes; i++) {
			g_sysTime += g_dt;
			timeStep(g_dt);
		}
	}

	function timeStep(deltat) {
		const nnx=g_NNx, psi=g_psi;

		const c = 137.0; // light speed in au
		const a = c*c*deltat;

		for (let i=0; i<nnx; i++) {
			let ip = i+1; if (ip>=nnx) ip = 0;
			let im = i-1; if (im<0) im = nnx-1;
			psi[0][i][0] += a*((-psi[1][ip][0]+psi[1][im][0])/2.0-psi[2][i][1]);
			psi[0][i][1] += a*((-psi[1][ip][1]+psi[1][im][1])/2.0+psi[2][i][0]);

			psi[1][i][0] += a*((-psi[0][ip][0]+psi[0][im][0])/2.0-psi[3][i][1]);
			psi[1][i][1] += a*((-psi[0][ip][1]+psi[0][im][1])/2.0+psi[3][i][0]);

			psi[2][i][0] += a*((psi[3][ip][0]-psi[3][im][0])/2.0-psi[0][i][1]);
			psi[2][i][1] += a*((psi[3][ip][1]-psi[3][im][1])/2.0+psi[0][i][0]);

			psi[3][i][0] += a*((psi[2][ip][0]-psi[2][im][0])/2.0-psi[1][i][1]);
			psi[3][i][1] += a*((psi[2][ip][1]-psi[2][im][1])/2.0+psi[1][i][0]);
		}
	}

	// --------------------  utilities

	function normDirac() {
		let s=0.0;
		for (let j=0; j<4; j++) {
			s += normSpinor(j);
		}
		return s;
	}

	function normSpinor(sp) {
		const nnx=g_NNx, psi=g_psi;
		let s=0.0;
		for (let i=0; i<nnx; i++) {
			s += (psi[sp][i][0]*psi[sp][i][0]+psi[sp][i][1]*psi[sp][i][1])*g_dx;
		}
		return s;
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( momentum )
		evolve:			timeEvolution,			// timeEvolution( nCalc )

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_dx, g_dt ]; },
		getNow:			function() { return [ g_sysTime, normDirac() ]; },
		getNormSpinor:	normSpinor,				// normSpinor( sp )
		getPsi:			function(sp,i) { return [ g_psi[sp][i][0], g_psi[sp][i][1] ]; },
	}

})(); // ====================  solveDiracEquation1D end  ====================


const js103 = (function(){ // ====================  js Module  ====================

	const theModule = solveDiracEquation1D;
	const psiColor = ['#ff4444', '#aaaa00', '#00bb00', '#4444ff'];
	const psiStr = ['alpha', 'beta', 'mue', 'nue'];

	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;

	let v_theme = 0;
	let v_momentum = 10.0;
	let v_nCalc = 20;

	let p_NNx, p_dx, p_dt; // = theModule.getSysParam();
	const p_NNy = 200, p_NNz = 200;

  let sysTime, normDirac;
  const spn = theModule.getNormSpinor;
  let spNormList = [ 0.0, 0.0, 0.0, 0.0 ];
	let nowData = [];
  let spinorRe = [];
  let spinorIm = [];

	let dispMode = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let inStepFlag = false;

  let breakFlag = false;
  let getFieldFlag = true;
  let fieldKind = 1;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_momentum );
			[ p_NNx, p_dx, p_dt ] = theModule.getSysParam();
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNx, p_NNy, p_NNz, p_dx, xCanvasSize, yCanvasSize, 450, 20 );
      getFieldFlag = true;
      fieldKind = 1;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc );
			inStepFlag = true;
		}

		draw( ctx, dispMode );

    spNormList = [ spn(0), spn(1), spn(2), spn(3) ];

    if ( getFieldFlag ) setFieldData( fieldKind );

		requestAnimationFrame(animate);
	}

  function setFieldData( fieldKind ) {
    if (fieldKind==1) {
      spinorRe = [];
      spinorIm = [];
			nowData = [ sysTime, normDirac, spNormList ];
      for (let sp=0; sp<4; sp++) {
        spinorRe[sp] = [];
        spinorIm[sp] = [];
        for (let i=0; i<p_NNx; i++) {
          let re, im;
          [ re, im ] = theModule.getPsi(sp,i)
          spinorRe[sp][i] = re;
          spinorIm[sp][i] = im;
        }
      }
    }
  }


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {

		let auLength, auTime, auEnergy, au2eV;
		[ auLength, auTime, auEnergy, au2eV ] = theModule.getAUinSI();
		[ sysTime, normDirac ] = theModule.getNow();

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		drawLines3D( ctx );

		ctx.fillStyle = "#888888";
		ctx.fillText(`x-box size = ${(p_NNx*p_dx)} (au)`, 20, yCanvasSize-30);
		ctx.fillText(`time = ${sysTime.toFixed(6)} (au)`, 20, yCanvasSize-10);

		for (let sp=0; sp<4; sp++) {
			ctx.fillStyle = psiColor[sp];
			const spNorm = theModule.getNormSpinor(sp).toFixed(8);
			ctx.fillText(`spinor(${sp}) norm = ${spNorm}`, 240, yCanvasSize-30-20*sp);
		}
		ctx.fillStyle = "#888888";
		ctx.fillText(`norm = ${normDirac.toFixed(8)}`, 240, yCanvasSize-10);

		document.getElementById("text_caption").innerHTML =
				"initial condition: |alpha|=1.0, |beta|=|mue|=|nue|= 0.0<br>";
	}


	function drawLines3D( ctx ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx;

		g3d.set3DRotateXY(0.0);
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow

		const mag = 7.5;
		for (let sp=0; sp<4; sp++) {
			const color = psiColor[sp];
			for (let i=0; i<nnx-1; i++) {
				let yi, zi; [ yi, zi ] = yzFunc( sp, i, mag );
				const x = i*g_dx, y = g3d.cy0-yi, z = g3d.cz0+zi;
				let yip, zip; [ yip, zip ] = yzFunc( sp, i+1, mag );
				const x1 = (i+1)*g_dx, y1 = g3d.cy0-yip, z1 = g3d.cz0+zip;
				const amp = Math.sqrt(yi*yi+zi*zi + yip*yip+zip*zip);
				const colr = (amp>0.05) ? color : '#444444';
				g3d.drawRotatedLine(ctx, x, y, z, x1, y1, z1, colr, sc, xp, yp );
			}

		}

		g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#666666";
		ctx.fillText("Ax="+(g3d.Ax*180/Math.PI).toFixed(1)+", Ay="+(g3d.Ay*180/Math.PI).toFixed(1), 10, 15);
		ctx.font = "16px 'sans-serif'";


		function yzFunc( sp, i, mag ) {
			let re, im; [ re, im ] = theModule.getPsi(sp,i);
			return [ mag*re, mag*im ];
		}

	};


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = 1;
	}

	function setMomentum() {  // -20.0 - 20.0
		v_momentum = Number(document.getElementById("range_momentum").value);
		document.getElementById("text_momentum").innerHTML = " " + v_momentum.toFixed(1);
		resetFlag = 1;
	}

	function viewHome() {
		g3d.setRotateAngle(-10,-10);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, normDirac, spNormList ];
  }

  function pygetFieldData() {
		fieldKind = 0;
    return [ nowData, spinorRe, spinorIm ];
  }


	// --------------------  public  --------------------

	return {
		main:				main,				// main()

		reset:				reset,				// reset()
		pause:				pause,				// pause()
		step:				step,				// step()

		setTheme:			setTheme,			// setTheme()
		setMomentum:		setMomentum,	// setMomentum()

		viewHome:			viewHome,			// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, normDirac, spNormList ]
    pygetFieldData: pygetFieldData, // pygetFieldData() : return [ nowData, spinorRe, spinorIm ]
	};

})(); // ====================  js103 end  ====================


const js = js103;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js103] solve Dirac Equation in free space 1D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas>
<br>

theme: spinor
    <span style="margin-right: 160px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>momentum </label>
<input type="range" id="range_momentum" min="-10.0" max="10.0" value="10.0" step="0.2"
style="width:300px" list="tickmarks_mpx" oninput="js.setMomentum()">
 = <span id="text_momentum"> 10.0</span> (au)
<datalist id="tickmarks_mpx"><option value="-10.0"><option value="-5.0"><option value="0.0">
<option value="5.0"><option value="10.0">
</datalist>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, norm, spNormList ] = eval_js( 'js.pygetData({})'.format(i) )
  sp_norm = [ int(sn*1000)/1000 for sn in spNormList ]
  print(f'i = {i:>2},  time = {sysTime:>10.6f} (au),  norm = {norm:>8.6f}, sp_norm : {sp_norm}')
  time.sleep(1)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# get spinor data: spinorRe, spinorIm  and save

import time
import numpy as np

# exec html-js code
exec_html_js()
print("-- start --")

# wait
wait_time = 40 # (s)
print(f"-- wait {wait_time} (sec) --")
time.sleep(wait_time)

# get spnor data  spinorRe, spinorIm --> SpRe, SpIm
print("-- get spinor data --")
[ nowData, spinorRe, spinorIm ] = eval_js( 'js.pygetFieldData()' )
[ sysTime, norm, spNormList ] = nowData
sp_norm = [ int(sn*1000)/1000 for sn in spNormList ]
print(f'got time = {sysTime:>10.6f} (au),  norm = {norm:>8.6f}, sp_norm : {sp_norm}')

# prepare numpy array for plot
SpRe = np.array(spinorRe)
SpIm = np.array(spinorIm)
print(f' SpRe shape :{SpRe.shape},  SpIm shape :{SpIm.shape}')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

print("-- save data : np.array([ spinorRe, spinorIm ]) --")
packed_data = np.array([ spinorRe, spinorIm ])

file_name = 'js103_data.npy'
np.save(file_name, packed_data)
print("-- saved data shape :", packed_data.shape, " --")

In [ ]:
# load data and unpack

import numpy as np

# load packed data : np_data = np.array([ spinorRe, spinorIm ])
loaded_data = np.load('js103_data.npy')

# set numpy array for plot
SpRe = loaded_data[0]  # real part of Spinor
SpIm = loaded_data[1]  # imaginary part of Spinor
print(f'loaded data shape : {loaded_data.shape},  SpRe shape :{SpRe.shape},  SpIm shape :{SpIm.shape}')

In [ ]:
# |Sp(x)| plot

import numpy as np
import matplotlib.pyplot as plt

nx = len(SpRe[0])
X = np.arange(0,nx) * 0.05

SPabs0 = SpRe[0]**2 + SpIm[0]**2
SPabs1 = SpRe[1]**2 + SpIm[1]**2
SPabs2 = SpRe[2]**2 + SpIm[2]**2
SPabs3 = SpRe[3]**2 + SpIm[3]**2

plt.plot(X,SPabs0, label='sp = 0')
plt.plot(X,SPabs1, label='sp = 1')
plt.plot(X,SPabs2, label='sp = 2')
plt.plot(X,SPabs3, label='sp = 3')
plt.legend()
plt.show()

In [ ]:
# Sp(x) phase plot

import numpy as np
import matplotlib.pyplot as plt

nx = len(SpRe[0])
X = np.arange(0,nx) * 0.05

Phase0 = np.arctan2(SpIm[0], SpRe[0]) * SPabs0
Phase1 = np.arctan2(SpIm[1], SpRe[1]) * SPabs1

plt.plot(X,Phase0, label='sp = 0')
plt.plot(X,Phase1, label='sp = 1')

plt.legend()
plt.show()

In [ ]:
# plot wave curve SpRe, spIm - matplotlib

import numpy as np
import matplotlib.pyplot as plt

n_sp, nx = SpRe.shape

ax = plt.figure().add_subplot(projection='3d')

# plot 0
sp = 0 # sp : 0, 1, 2, 3
X = np.arange(0,nx) * 0.05
Y = SpRe[sp]
Z = SpIm[sp]
ax.plot(X, Y, Z, label='sp = 0')

# plot 1
sp = 1
Y = SpRe[sp]
Z = SpIm[sp]
ax.plot(X, Y, Z, label='sp = 1')

# plot 2
sp = 2
Y = SpRe[sp]
Z = SpIm[sp]
ax.plot(X, Y, Z, label='sp = 2')

ax.legend()
plt.show()

In [ ]:
# plot wave curve SpRe, spIm - plotly

import numpy as np
import plotly.graph_objects as go

n_sp, nx = SpRe.shape # n_sp: number of spinor = 4, nx: number of spinor data, x = ni*0.05

# plot 0
sp = 0
X = np.arange(0,nx) * 0.05
Y = SpRe[sp]
Z = SpIm[sp]
fig = go.Figure(data=go.Scatter3d(x=X, y=Y, z=Z, mode='lines', name='sp = 0'))

# plot 1
sp = 1
Y = SpRe[sp]
Z = SpIm[sp]
fig.add_trace(go.Scatter3d(x=X, y=Y, z=Z, mode='lines', name='sp = 1'))

# plot 2
sp = 2
Y = SpRe[sp]
Z = SpIm[sp]
fig.add_trace(go.Scatter3d(x=X, y=Y, z=Z, mode='lines', name='sp = 2'))

fig.update_layout(
  title='spinor wave curve',
  width=1000, height=700,
  scene = {
        'camera_eye': {"x": 0, "y": -1, "z": 0.5},
        "aspectratio": {"x": 1, "y": 0.5, "z": 0.5}
    }
)

fig.show()

In [11]:
# def normDirac(), normSpinor(sp)

""" js code

	function normDirac() {
		let s=0.0;
		for (let j=0; j<4; j++) {
			s += normSpinor(j);
		}
		return s;
	}

	function normSpinor(sp) {
		const nnx=g_NNx, psi=g_psi;
		let s=0.0;
		for (let i=0; i<nnx; i++) {
			s += (psi[sp][i][0]*psi[sp][i][0]+psi[sp][i][1]*psi[sp][i][1])*g_dx;
		}
		return s;
	}

  // dx=0.05, ( psi[sp][i][0]--> SpRe[sp][i], psi[sp][i][1] --> SpIm[sp][i] )
"""

nx =len(SpRe[0])
dx = 0.05

def normDirac():
    s = 0.0
    for j in range(4):
        s += normSpinor(j)
    return s

def normSpinor(sp):
    s = 0.0
    for i in range(nx):
      s += SpRe[sp][i]**2 + SpIm[sp][i]**2 * dx
    return s


In [ ]:
# prompt: calc norm of Spinor using normDirac() and normSpinor()

import numpy as np

nx =len(SpRe[0])
dx = 0.05

def normDirac():
    s = 0.0
    for j in range(4):
        s += normSpinor(j)
    return s

def normSpinor(sp):
    s = 0.0
    for i in range(nx):
        s += (SpRe[sp][i]**2 + SpIm[sp][i]**2) * dx
    return s

# Example usage (assuming SpRe, SpIm, nx, and dx are defined as in your original code)
dirac_norm = normDirac()
print(f"Dirac Norm: {dirac_norm}")

for sp in range(4):
    spinor_norm = normSpinor(sp)
    print(f"Spinor Norm (sp={sp}): {spinor_norm}")